In [5]:
from datetime import datetime, timedelta
from pandas import DataFrame
from models.Payload import REGION, RegPayload, SecPayload
from models.Source import DataSource as SecApi
from utils.transforms import str2date
from urllib.parse import urlencode as url
#cspell:disable

#input
region:REGION='Valparaiso'
#total regional clients
users = SecApi()
q = users.request('reg_clients',RegPayload(region=region))
clients = int(q[0]['CLIENTES'])
print(f"usuarios regionales: {clients:,.0f} clientes")

#data
curr = SecApi()
curr.request('affected_detail',SecPayload()) 
curr_df=curr.dataframe(FECHA_INT_STR=str2date)
curr_df= curr_df[curr_df['NOMBRE_REGION']==region].drop(columns=['HORA','DIA','MES','ANHO'])
affected = curr_df['CLIENTES_AFECTADOS'].sum()

print('usuarios sin servicio :',f'{affected:,.0f}',' clientes',f'{affected/clients:.2%}')

res:DataFrame = curr_df.groupby(['NOMBRE_COMUNA','NOMBRE_EMPRESA','FECHA_INT_STR'],as_index=False)['CLIENTES_AFECTADOS'].sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)

#cspell:disable
res:DataFrame = res.rename(columns={
    'FECHA_INT_STR':'INCIDENTE',
    'CLIENTES_AFECTADOS':'SIN_SERVICIO',
    'NOMBRE_EMPRESA':'DISTRIBUIDORA',
    'NOMBRE_COMUNA':'COMUNA'
    })

#days w/o service
res['TIEMPO_SIN_SERVICIO'] = res['INCIDENTE'].apply(lambda event_date: datetime.now()-event_date+timedelta(days=1) )
res

usuarios regionales: 780,563 clientes
usuarios sin servicio : 705  clientes 0.09%


,COMUNA,DISTRIBUIDORA,INCIDENTE,SIN_SERVICIO,TIEMPO_SIN_SERVICIO
8,Viña del Mar,CHILQUINTA,2024-02-19,513,184 days 16:17:16.509637
1,Cartagena,CHILQUINTA,2024-08-19,100,2 days 16:17:16.509637
4,Quilpue,CHILQUINTA,2024-02-19,83,184 days 16:17:16.509637
7,Viña del Mar,CGE,2024-08-20,4,1 days 16:17:16.509637
0,Cartagena,CGE,2024-08-20,1,1 days 16:17:16.509637
2,La Ligua,CGE,2024-08-20,1,1 days 16:17:16.509637
3,Papudo,CGE,2024-08-20,1,1 days 16:17:16.509637
5,San Antonio,CGE,2024-08-20,1,1 days 16:17:16.509637
6,Valparaiso,CGE,2024-08-20,1,1 days 16:17:16.509637


In [6]:
res_by_city:DataFrame = curr_df.groupby(['NOMBRE_COMUNA'],as_index=False)['CLIENTES_AFECTADOS']\
    .sum().sort_values(['CLIENTES_AFECTADOS'],ascending=False)
res_by_city

,NOMBRE_COMUNA,CLIENTES_AFECTADOS
6,Viña del Mar,517
0,Cartagena,101
3,Quilpue,83
1,La Ligua,1
2,Papudo,1
4,San Antonio,1
5,Valparaiso,1
